# 1. Introdução

Este projeto tem como objetivo criar um algoritmo que identifique os funcionários da Enron que podem ter cometido fraude baseando-se no conjunto de dados público entitulado "Enron financial and email".

Em 2000, Enron era uma das maiores empresas dos Estados Unidos. Já em 2002, ela colapsou e quebrou devido a uma fraude que envolveu grande parte da corporação. Resultando em uma investigação federal, muitos dados que são normalmente confidenciais, se tornaram públicos, incluindo dezenas de milhares de e-mails e detalhes financeiros para os executivos dos mais altos níveis da empresa.

Este documento visa explicar minha linha de raciocínio e responder as questões, indagadas pela Udacity, para avaliação deste projeto. As perguntas encontram-se neste [link.](https://docs.google.com/document/d/1NDgi1PrNJP7WTbfSUuRUnz8yzs5nGVTSzpO7oeNTEWA/pub?embedded=true)

# 2. Conjunto de dados

Os dados foram disponibilizados em um dicionário, onde cada par chave-valor corresponde a uma pessoa. A chave do dicionário é o nome da pessoa, e o valor é outro dicionário, que contém o nome de todos os atributos e seus valores para aquela pessoa. Os atributos nos dados possuem basicamente três tipos: atributos financeiros, de email e rótulos POI (pessoa de interesse).

>**atributos financeiros:** ['salary', 'deferral_payments', 'total_payments', 'loan_advances', 'bonus', 'restricted_stock_deferred', 'deferred_income', 'total_stock_value', 'expenses', 'exercised_stock_options', 'other', 'long_term_incentive', 'restricted_stock', 'director_fees'] (todos em dólares americanos (USD))

>**atributos de email:** ['to_messages', 'email_address', 'from_poi_to_this_person', 'from_messages', 'from_this_person_to_poi', 'shared_receipt_with_poi'] (as unidades aqui são geralmente em número de emails; a exceção notável aqui é o atributo ‘email_address’, que é uma string)

>**rótulo POI:** [‘poi’] (atributo objetivo lógico (booleano), representado como um inteiro)

# 3. Seleção de atributos (features)

Antes de selecionar os atributos iniciais a serem usados, deve-se primeiro realizar uma análise sobre a situação dos dados disponibilizados. O objetivo dessa análise é descobrir como os dados estão estruturados, se existem dados faltantes, problemas com os tipos de dados ou algum outro tipo de problema que inviabilize o uso de alguma feature inicialmente.

## 3.1. Exploração dos dados 

>Importando as bibliotecas necessárias para a análise

In [28]:
import sys
import pickle
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings('ignore')

>Carregando os dados...

In [29]:
with open("final_project_dataset.pkl", "rb") as data_file:
    data_dict = pickle.load(data_file)

df = pd.DataFrame.from_dict(data_dict, orient='index')

>Tamanho dos dados...

In [30]:
print("{0} linhas, {1} colunas".format(df.shape[0],df.shape[1]))

146 linhas, 21 colunas


>Visualizando os dados...

In [31]:
df.head(10)

,salary,to_messages,deferral_payments,total_payments,loan_advances,bonus,email_address,restricted_stock_deferred,deferred_income,total_stock_value,...,from_poi_to_this_person,exercised_stock_options,from_messages,other,from_this_person_to_poi,poi,long_term_incentive,shared_receipt_with_poi,restricted_stock,director_fees
ALLEN PHILLIP K,201955,2902,2869717,4484442,NaN,4175000,phillip.allen@enron.com,-126027,-3081055,1729541,...,47,1729541,2195,152,65,False,304805,1407,126027,NaN
BADUM JAMES P,NaN,NaN,178980,182466,NaN,NaN,NaN,NaN,NaN,257817,...,NaN,257817,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN
BANNANTINE JAMES M,477,566,NaN,916197,NaN,NaN,james.bannantine@enron.com,-560222,-5104,5243487,...,39,4046157,29,864523,0,False,NaN,465,1757552,NaN
BAXTER JOHN C,267102,NaN,1295738,5634343,NaN,1200000,NaN,NaN,-1386055,10623258,...,NaN,6680544,NaN,2660303,NaN,False,1586055,NaN,3942714,NaN
BAY FRANKLIN R,239671,NaN,260455,827696,NaN,400000,frank.bay@enron.com,-82782,-201641,63014,...,NaN,NaN,NaN,69,NaN,False,NaN,NaN,145796,NaN
BAZELIDES PHILIP J,80818,NaN,684694,860136,NaN,NaN,NaN,NaN,NaN,1599641,...,NaN,1599641,NaN,874,NaN,False,93750,NaN,NaN,NaN
BECK SALLY W,231330,7315,NaN,969068,NaN,700000,sally.beck@enron.com,NaN,NaN,126027,...,144,NaN,4343,566,386,False,NaN,2639,126027,NaN
BELDEN TIMOTHY N,213999,7991,2144013,5501630,NaN,5249999,tim.belden@enron.com,NaN,-2334434,1110705,...,228,953136,484,210698,108,True,NaN,5521,157569,NaN
BELFER ROBERT,NaN,NaN,-102500,102500,NaN,NaN,NaN,44093,NaN,-44093,...,NaN,3285,NaN,NaN,NaN,False,NaN,NaN,NaN,3285
BERBERIAN DAVID,216582,NaN,NaN,228474,NaN,NaN,david.berberian@enron.com,NaN,NaN,2493616,...,NaN,1624396,NaN,NaN,NaN,False,NaN,NaN,869220,NaN


>Vizualizando tipos de dados e dados faltantes...

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 146 entries, ALLEN PHILLIP K to YEAP SOON
Data columns (total 21 columns):
salary                       146 non-null object
to_messages                  146 non-null object
deferral_payments            146 non-null object
total_payments               146 non-null object
loan_advances                146 non-null object
bonus                        146 non-null object
email_address                146 non-null object
restricted_stock_deferred    146 non-null object
deferred_income              146 non-null object
total_stock_value            146 non-null object
expenses                     146 non-null object
from_poi_to_this_person      146 non-null object
exercised_stock_options      146 non-null object
from_messages                146 non-null object
other                        146 non-null object
from_this_person_to_poi      146 non-null object
poi                          146 non-null bool
long_term_incentive          146 non-null object


Primeiro problema encontrado: Muitos atribuitos faltantes estão com os valores 'NAN' no formato texto. Corrigir este problema pois pode influenciar na estatística. 

In [33]:
df = df.replace('NaN', np.NaN)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 146 entries, ALLEN PHILLIP K to YEAP SOON
Data columns (total 21 columns):
salary                       95 non-null float64
to_messages                  86 non-null float64
deferral_payments            39 non-null float64
total_payments               125 non-null float64
loan_advances                4 non-null float64
bonus                        82 non-null float64
email_address                111 non-null object
restricted_stock_deferred    18 non-null float64
deferred_income              49 non-null float64
total_stock_value            126 non-null float64
expenses                     95 non-null float64
from_poi_to_this_person      86 non-null float64
exercised_stock_options      102 non-null float64
from_messages                86 non-null float64
other                        93 non-null float64
from_this_person_to_poi      86 non-null float64
poi                          146 non-null bool
long_term_incentive          66 non-null float6

Agora vemos que somente o atributo 'poi' está completo.

>Quantas variáveis faltante cada funcionário possui?

In [34]:
df.isnull().sum(axis=1).sort_values(ascending=False)

LOCKHART EUGENE E                20
GRAMM WENDY L                    18
WROBEL BRUCE                     18
WHALEY DAVID A                   18
THE TRAVEL AGENCY IN THE PARK    18
WAKEHAM JOHN                     17
WODRASKA JOHN                    17
CLINE KENNETH W                  17
GILLIS JOHN                      17
SCRIMSHAW MATTHEW                17
SAVAGE FRANK                     17
MENDELSOHN JOHN                  16
YEAP SOON                        16
CHRISTODOULOU DIOMEDES           16
PEREIRA PAULO V. FERRAZ          16
BLAKE JR. NORMAN P               16
LOWRY CHARLES P                  16
CHAN RONNIE                      16
MEYER JEROME J                   16
GATHMANN WILLIAM D               16
WINOKUR JR. HERBERT S            16
FUGH JOHN L                      16
URQUHART JOHN A                  16
NOLES JAMES L                    15
WALTERS GARETH W                 15
BADUM JAMES P                    15
LEMAISTRE CHARLES                15
DUNCAN JOHN H               

Vemos que a pessoa **LOCKHART EUGENE E** não possui nenhum atributo além do 'poi'. Então ele deve ser retirado da base já que não possui nenhuma informação.

In [35]:
df.drop('LOCKHART EUGENE E', inplace=True)

>**THE TRAVEL AGENCY IN THE PARK** não parece ser uma pessoa. Quais os atributos que possui?

In [36]:
df.loc['THE TRAVEL AGENCY IN THE PARK']

salary                          NaN
to_messages                     NaN
deferral_payments               NaN
total_payments               362096
loan_advances                   NaN
bonus                           NaN
email_address                   NaN
restricted_stock_deferred       NaN
deferred_income                 NaN
total_stock_value               NaN
expenses                        NaN
from_poi_to_this_person         NaN
exercised_stock_options         NaN
from_messages                   NaN
other                        362096
from_this_person_to_poi         NaN
poi                           False
long_term_incentive             NaN
shared_receipt_with_poi         NaN
restricted_stock                NaN
director_fees                   NaN
Name: THE TRAVEL AGENCY IN THE PARK, dtype: object

Devemos excluir **THE TRAVEL AGENCY IN THE PARK**. Pois não se trata de uma pessoa.

In [37]:
df.drop('THE TRAVEL AGENCY IN THE PARK', inplace=True)

>Quantos e quais funcionários são classificados como POI?

In [38]:
df.poi.value_counts()

False    126
True      18
Name: poi, dtype: int64

In [39]:
df.query("poi==True")

,salary,to_messages,deferral_payments,total_payments,loan_advances,bonus,email_address,restricted_stock_deferred,deferred_income,total_stock_value,...,from_poi_to_this_person,exercised_stock_options,from_messages,other,from_this_person_to_poi,poi,long_term_incentive,shared_receipt_with_poi,restricted_stock,director_fees
BELDEN TIMOTHY N,213999.0,7991.0,2144013.0,5501630.0,NaN,5249999.0,tim.belden@enron.com,NaN,-2334434.0,1110705.0,...,228.0,953136.0,484.0,210698.0,108.0,True,NaN,5521.0,157569.0,NaN
BOWEN JR RAYMOND M,278601.0,1858.0,NaN,2669589.0,NaN,1350000.0,raymond.bowen@enron.com,NaN,-833.0,252055.0,...,140.0,NaN,27.0,1621.0,15.0,True,974293.0,1593.0,252055.0,NaN
CALGER CHRISTOPHER F,240189.0,2598.0,NaN,1639297.0,NaN,1250000.0,christopher.calger@enron.com,NaN,-262500.0,126027.0,...,199.0,NaN,144.0,486.0,25.0,True,375304.0,2188.0,126027.0,NaN
CAUSEY RICHARD A,415189.0,1892.0,NaN,1868758.0,NaN,1000000.0,richard.causey@enron.com,NaN,-235000.0,2502063.0,...,58.0,NaN,49.0,307895.0,12.0,True,350000.0,1585.0,2502063.0,NaN
COLWELL WESLEY,288542.0,1758.0,27610.0,1490344.0,NaN,1200000.0,wes.colwell@enron.com,NaN,-144062.0,698242.0,...,240.0,NaN,40.0,101740.0,11.0,True,NaN,1132.0,698242.0,NaN
DELAINEY DAVID W,365163.0,3093.0,NaN,4747979.0,NaN,3000000.0,david.delainey@enron.com,NaN,NaN,3614261.0,...,66.0,2291113.0,3069.0,1661.0,609.0,True,1294981.0,2097.0,1323148.0,NaN
FASTOW ANDREW S,440698.0,NaN,NaN,2424083.0,NaN,1300000.0,andrew.fastow@enron.com,NaN,-1386055.0,1794412.0,...,NaN,NaN,NaN,277464.0,NaN,True,1736055.0,NaN,1794412.0,NaN
GLISAN JR BEN F,274975.0,873.0,NaN,1272284.0,NaN,600000.0,ben.glisan@enron.com,NaN,NaN,778546.0,...,52.0,384728.0,16.0,200308.0,6.0,True,71023.0,874.0,393818.0,NaN
HANNON KEVIN P,243293.0,1045.0,NaN,288682.0,NaN,1500000.0,kevin.hannon@enron.com,NaN,-3117011.0,6391065.0,...,32.0,5538001.0,32.0,11350.0,21.0,True,1617011.0,1035.0,853064.0,NaN
HIRKO JOSEPH,NaN,NaN,10259.0,91093.0,NaN,NaN,joe.hirko@enron.com,NaN,NaN,30766064.0,...,NaN,30766064.0,NaN,2856.0,NaN,True,NaN,NaN,NaN,NaN


>Dos funcionários, quem mais recebeu dinheiro?

In [40]:
df.sort_values(by='total_payments', ascending=False).head(10)

,salary,to_messages,deferral_payments,total_payments,loan_advances,bonus,email_address,restricted_stock_deferred,deferred_income,total_stock_value,...,from_poi_to_this_person,exercised_stock_options,from_messages,other,from_this_person_to_poi,poi,long_term_incentive,shared_receipt_with_poi,restricted_stock,director_fees
TOTAL,26704229.0,NaN,32083396.0,309886585.0,83925000.0,97343619.0,NaN,-7576788.0,-27992891.0,434509511.0,...,NaN,311764000.0,NaN,42667589.0,NaN,False,48521928.0,NaN,130322299.0,1398517.0
LAY KENNETH L,1072321.0,4273.0,202911.0,103559793.0,81525000.0,7000000.0,kenneth.lay@enron.com,NaN,-300000.0,49110078.0,...,123.0,34348384.0,36.0,10359729.0,16.0,True,3600000.0,2411.0,14761694.0,NaN
FREVERT MARK A,1060932.0,3275.0,6426990.0,17252530.0,2000000.0,2000000.0,mark.frevert@enron.com,NaN,-3367011.0,14622185.0,...,242.0,10433518.0,21.0,7427621.0,6.0,False,1617011.0,2979.0,4188667.0,NaN
BHATNAGAR SANJAY,NaN,523.0,NaN,15456290.0,NaN,NaN,sanjay.bhatnagar@enron.com,15456290.0,NaN,NaN,...,0.0,2604490.0,29.0,137864.0,1.0,False,NaN,463.0,-2604490.0,137864.0
LAVORATO JOHN J,339288.0,7259.0,NaN,10425757.0,NaN,8000000.0,john.lavorato@enron.com,NaN,NaN,5167144.0,...,528.0,4158995.0,2585.0,1552.0,411.0,False,2035380.0,3962.0,1008149.0,NaN
SKILLING JEFFREY K,1111258.0,3627.0,NaN,8682716.0,NaN,5600000.0,jeff.skilling@enron.com,NaN,NaN,26093672.0,...,88.0,19250000.0,108.0,22122.0,30.0,True,1920000.0,2042.0,6843672.0,NaN
MARTIN AMANDA K,349487.0,1522.0,85430.0,8407016.0,NaN,NaN,a..martin@enron.com,NaN,NaN,2070306.0,...,8.0,2070306.0,230.0,2818454.0,0.0,False,5145434.0,477.0,NaN,NaN
BAXTER JOHN C,267102.0,NaN,1295738.0,5634343.0,NaN,1200000.0,NaN,NaN,-1386055.0,10623258.0,...,NaN,6680544.0,NaN,2660303.0,NaN,False,1586055.0,NaN,3942714.0,NaN
BELDEN TIMOTHY N,213999.0,7991.0,2144013.0,5501630.0,NaN,5249999.0,tim.belden@enron.com,NaN,-2334434.0,1110705.0,...,228.0,953136.0,484.0,210698.0,108.0,True,NaN,5521.0,157569.0,NaN
DELAINEY DAVID W,365163.0,3093.0,NaN,4747979.0,NaN,3000000.0,david.delainey@enron.com,NaN,NaN,3614261.0,...,66.0,2291113.0,3069.0,1661.0,609.0,True,1294981.0,2097.0,1323148.0,NaN


Descobrimos mais um problema. **TOTAL** não é um funcionário, e sim um registro que é um somatório de todos os pagamentos feitos a funcionários. Deve ser excluído.

In [41]:
df.drop('TOTAL', inplace=True)

## 3.2. Escolhendos os atributos 

>Como este é um caso de fraudes, então uma forma de selecionar os atributos é selecionando aqueles que possium mais outliers. Quais as variáveis que mais possuem outliers?

Obs.: Método utilizado foi o IQR(interquartile range), que pode ser encontrado no [link](http://colingorrie.github.io/outlier-detection.html)

In [42]:
columns = df.columns.values
columns=np.delete(columns,6) #Removi o atributo email_address pois estava dando problema no cálculo

Q1 = df[columns].quantile(0.25)
Q3 = df[columns].quantile(0.75)
IQR = Q3 - Q1
n_outliers = ((df[columns] < (Q1 - 1.5 * IQR)) | (df[columns] > (Q3 + 1.5 * IQR))).sum()
n_outliers.sort_values(ascending=False)

total_stock_value            21
poi                          18
from_messages                17
restricted_stock             14
from_this_person_to_poi      13
other                        11
exercised_stock_options      11
from_poi_to_this_person      11
total_payments               10
bonus                        10
salary                        9
to_messages                   7
long_term_incentive           7
deferral_payments             6
deferred_income               5
director_fees                 4
expenses                      3
restricted_stock_deferred     2
shared_receipt_with_poi       2
loan_advances                 0
dtype: int64

 Baseado no resultado, Decidi usar os 10 atributos que mais possuem outliers, com exceção do atributo **poi**

In [43]:
feature_list = ['poi', 'total_stock_value', 'from_messages', 'restricted_stock', 'from_this_person_to_poi', 'other', 'exercised_stock_options', 'from_poi_to_this_person', 'total_payments', 'bonus', 'salary']


Obs.: Nas próximas sessões iremos usar o algoritmo de seleção de features e comparar com nossa escolha inicial.

## 3.3. Criando novos atributos 

Todos os dados do dataset são valores brutos. As vezes, saber a porcentagem que o bonus ou salário em cima do total de pagamentos, é bem mais informativo. Seguindo o mesmo raciocínio, podemos falar do total de emails recebidos e enviado para POIs. Por isso criei as variáveis abaixo:

1. perc_salary: percentual do salario em relação ao total pago.
2. perc_bonus: percentual de bonus em relação ao total pago
3. perc_messages_send_to_poi: percentual de mensagens enviadas para POIs, em relação ao total
4. perc_messages_rec_from_poi: percentual de mensagens recebidas de POIs, em relação ao total

In [44]:
df['perc_salary'] = df['salary']/df['total_payments']
df['perc_bonus'] = df['bonus']/df['total_payments']
df['perc_messages_send_to_poi'] = df['from_this_person_to_poi']/df['to_messages']
df['perc_messages_rec_from_poi'] = df['from_poi_to_this_person']/df['from_messages']
df.head()

,salary,to_messages,deferral_payments,total_payments,loan_advances,bonus,email_address,restricted_stock_deferred,deferred_income,total_stock_value,...,from_this_person_to_poi,poi,long_term_incentive,shared_receipt_with_poi,restricted_stock,director_fees,perc_salary,perc_bonus,perc_messages_send_to_poi,perc_messages_rec_from_poi
ALLEN PHILLIP K,201955.0,2902.0,2869717.0,4484442.0,NaN,4175000.0,phillip.allen@enron.com,-126027.0,-3081055.0,1729541.0,...,65.0,False,304805.0,1407.0,126027.0,NaN,0.045035,0.930997,0.022398,0.021412
BADUM JAMES P,NaN,NaN,178980.0,182466.0,NaN,NaN,NaN,NaN,NaN,257817.0,...,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BANNANTINE JAMES M,477.0,566.0,NaN,916197.0,NaN,NaN,james.bannantine@enron.com,-560222.0,-5104.0,5243487.0,...,0.0,False,NaN,465.0,1757552.0,NaN,0.000521,NaN,0.000000,1.344828
BAXTER JOHN C,267102.0,NaN,1295738.0,5634343.0,NaN,1200000.0,NaN,NaN,-1386055.0,10623258.0,...,NaN,False,1586055.0,NaN,3942714.0,NaN,0.047406,0.212980,NaN,NaN
BAY FRANKLIN R,239671.0,NaN,260455.0,827696.0,NaN,400000.0,frank.bay@enron.com,-82782.0,-201641.0,63014.0,...,NaN,False,NaN,NaN,145796.0,NaN,0.289564,0.483269,NaN,NaN


>Uma segunda lista de atributos sera criada com essas novas features para comparação dos resultados com as features sem os novos atributos

In [45]:
feature_list_1 = ['poi', 'total_stock_value', 'from_messages', 'restricted_stock', 'from_this_person_to_poi', 'other', 'exercised_stock_options', 'from_poi_to_this_person', 'total_payments', 'bonus', 'salary', 'perc_salary','perc_bonus','perc_messages_send_to_poi','perc_messages_rec_from_poi']

# 4. Remoção de outliers

>Na sessão anterior, acabamos removendos os outliers necessários **('TOTAL', 'THE TRAVEL AGENCY IN THE PARK', 'LOCKHART EUGENE E')**. Fora isso, como o caso é de fraude, o resto dos outliers não devem ser removidos

# 5. Testando os Classificadores

Escolhi 3 classificadores para realizar esse teste. **Naive Bayes**, **Random Forest**, **Decicion Tree** e **K-means**. Inicialmente vamos testar o desempenho de cada um deles com as features escolhidas no final do item 3 desta análise. Após isso, vamos reavaliar as escolhas das features e fazer um tunning para ver o que consiguimos melhorar no desempenho destes algoritmos. 

## 5.1. Pré-processamentos dos dados

In [46]:
from feature_format import featureFormat, targetFeatureSplit
from tester import dump_classifier_and_data, main
from sklearn.model_selection import train_test_split

>Formatando os dados no formato que os classificadores utilizam 

In [47]:
df.fillna('NaN', inplace=True)
my_dataset = df.to_dict('index')
#Sem novos atributos
data = featureFormat(my_dataset, feature_list, sort_keys = False)
labels, features = targetFeatureSplit(data)

#Com novos atributos
data_1 = featureFormat(my_dataset, feature_list_1, sort_keys = False)
labels_1, features_1 = targetFeatureSplit(data_1)

>Separando o conjunto de dados para treinamento e teste

In [48]:
features_train, features_test, labels_train, labels_test = \
    train_test_split(features, labels, test_size=0.3, random_state=42)

## 5.2. Classificadores

Para testar os classificadores vamos usar s funções dump_classifier_and_data, main do arquivo tester, que foi disponibilizado pela Udacity

### 5.2.1. Naive Bayes

>Importando a biblioteca. Criando o classificador e testando.

In [49]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
dump_classifier_and_data(clf, my_dataset, feature_list)
main()

GaussianNB(priors=None)
	Accuracy: 0.84807	Precision: 0.38668	Recall: 0.23800	F1: 0.29465	F2: 0.25783
	Total predictions: 15000	True positives:  476	False positives:  755	False negatives: 1524	True negatives: 12245



In [50]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
dump_classifier_and_data(clf, my_dataset, feature_list_1)
main()

GaussianNB(priors=None)
	Accuracy: 0.84807	Precision: 0.38668	Recall: 0.23800	F1: 0.29465	F2: 0.25783
	Total predictions: 15000	True positives:  476	False positives:  755	False negatives: 1524	True negatives: 12245



Não houve diferença entre de resultados para as novas features

### 5.2.2. Random Forest

>Importando a biblioteca. Criando o classificador e testando.

In [51]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
dump_classifier_and_data(clf, my_dataset, feature_list)
main()

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
	Accuracy: 0.85680	Precision: 0.38889	Recall: 0.12950	F1: 0.19430	F2: 0.14943
	Total predictions: 15000	True positives:  259	False positives:  407	False negatives: 1741	True negatives: 12593



In [52]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
dump_classifier_and_data(clf, my_dataset, feature_list_1)
main()

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
	Accuracy: 0.85913	Precision: 0.40989	Recall: 0.12850	F1: 0.19566	F2: 0.14895
	Total predictions: 15000	True positives:  257	False positives:  370	False negatives: 1743	True negatives: 12630



Houve uma pequena melhora com o uso das novas features. F1(sem): 0.19430 < F1(com): 0.19566

### 5.2.3. Decision Tree

>Importando a biblioteca. Criando o classificador e testando.

In [53]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
dump_classifier_and_data(clf, my_dataset, feature_list)
main()

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')
	Accuracy: 0.79267	Precision: 0.23032	Recall: 0.23700	F1: 0.23361	F2: 0.23563
	Total predictions: 15000	True positives:  474	False positives: 1584	False negatives: 1526	True negatives: 11416



In [54]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
dump_classifier_and_data(clf, my_dataset, feature_list_1)
main()

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')
	Accuracy: 0.79100	Precision: 0.23690	Recall: 0.25550	F1: 0.24585	F2: 0.25155
	Total predictions: 15000	True positives:  511	False positives: 1646	False negatives: 1489	True negatives: 11354



Houve uma pequena melhora com o uso das novas features. F1(sem): 0.23361 < F1(com): 0.24585

### 5.2.4. K-means

>Importando a biblioteca. Criando o classificador e testando.

In [55]:
from sklearn.cluster import KMeans
clf = KMeans(n_clusters=2, random_state=0).fit(features_train)
dump_classifier_and_data(clf, my_dataset, feature_list)
main()

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=2, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=0, tol=0.0001, verbose=0)
	Accuracy: 0.85533	Precision: 0.31360	Recall: 0.07150	F1: 0.11645	F2: 0.08456
	Total predictions: 15000	True positives:  143	False positives:  313	False negatives: 1857	True negatives: 12687



In [56]:
from sklearn.cluster import KMeans
clf = KMeans(n_clusters=2, random_state=0).fit(features_train)
dump_classifier_and_data(clf, my_dataset, feature_list_1)
main()

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=2, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=0, tol=0.0001, verbose=0)
	Accuracy: 0.85533	Precision: 0.31360	Recall: 0.07150	F1: 0.11645	F2: 0.08456
	Total predictions: 15000	True positives:  143	False positives:  313	False negatives: 1857	True negatives: 12687



Não houve melhoras nesse caso.

### 5.2.5. Resultados 

>**Naive Bayes:** Accuracy: 0.84807	Precision: 0.38668	Recall: 0.23800	F1: 0.29465	F2: 0.25783

>**Random Forest:** Accuracy: 0.86113	Precision: 0.43118	Recall: 0.13000	F1: 0.19977	F2: 0.15111

>**Decision Tree:** Accuracy: 0.79193	Precision: 0.23747	Recall: 0.25350	F1: 0.24522	F2: 0.25012

>**K-means: Accuracy:** Accuracy: 0.85533	Precision: 0.31360	Recall: 0.07150	F1: 0.11645	F2: 0.084568

Dentre os classificadores testados, o melhor foi o **Naive Bayes**. E como houve melhores dos resultados com as novas features em alguns classificadores, resolvi mantê-los.

## 5.3. Reseleção dos atributos, tunning dos classificadores e Validação

Agora vamos realizar os passos abaixo para cada classificador para ver o que conseguimos melhorar

>a) Normalização dos dados, utilizando StandardScaler

>b) Redução de dimensionalidade dos dados, utilizando PCA;

>c) Seleção das Features: o algoritmo SelectKBest( seleciona os k melhores atributos baseados em testes estatísticos univariados) em conjunto com o GridSearchCV, podemos otimizar a quantidade de atributos para cada classificador selecionado;

>d) Otimização dos parâmetros do classificador: processo para selecionar os valores para os parâmetros de um modelo que maximizam a precisão do modelo, e para isso vamos usar o GridSearchCV; 

>e) Validação cruzada: Quando um modelo apresenta uma grande quantidade de parâmetros e quanto menos restrições colocarmos, maior a probabilidade de encontrarmos um super ajustamento(overfitting). Para evitar este problema existe o processo de validação. Há diversos tipos de validação, um dos mais utilizados na prática corresponde a dividir os seus dados em três bases, base de treino, validação e teste. No nosso caso vamos utilizar o **StratifiedShuffleSplit**. Na função, o classificador é ajustado ao conjunto de treinamento e, em seguida, os valores previstos são produzidos, pontuados e relatados para o conjunto de treinamento. Este método é empregado, pois preserva uma porcentagem de amostras para cada classe. Isso é importante, pois os POIs representam uma pequena parte do conjunto de dados;

In [57]:
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV, StratifiedShuffleSplit
from sklearn.feature_selection import SelectKBest

>Carregando lista com todas os atributos

In [58]:
features_list = ['poi',
                 'exercised_stock_options',
                 'total_stock_value',
                 'bonus',
                 'salary',
                 'deferred_income',
                 'long_term_incentive',
                 'restricted_stock',
                 'total_payments',
                 'shared_receipt_with_poi',
                 'loan_advances',
                 'expenses',
                 'from_poi_to_this_person',
                 'other',
                 'from_this_person_to_poi',
                 'director_fees',
                 'to_messages',
                 'deferral_payments',
                 'from_messages',
                 'restricted_stock_deferred',
                 'perc_salary',
                 'perc_bonus',
                 'perc_messages_send_to_poi',
                 'perc_messages_rec_from_poi'
                ]

In [59]:
my_dataset = df.to_dict('index')
data = featureFormat(my_dataset, features_list, sort_keys = False)
labels, features = targetFeatureSplit(data)

In [60]:
features_train, features_test, labels_train, labels_test = \
    train_test_split(features, labels, test_size=0.3, random_state=42)

### 5.3.1. Naive Bayes 

>Pipeline de execução para o classificador

In [61]:
#No caso de Naive Bayes, não foi utilizado o GridSearchCV, para otimização dos kernels params. 
pipe = Pipeline([
        ('scaler', preprocessing.StandardScaler()),
        ('reducer', PCA(random_state=42)),
        ('selector', SelectKBest()),
        ('classifier', GaussianNB())
    ])

In [62]:
param_grid = { 
    'selector__k': [2,4,6,8,10,12,14,16,18,20,'all']
}

In [63]:
sss = StratifiedShuffleSplit(n_splits=10, test_size=0.2, random_state=42)

In [64]:
grid_search = GridSearchCV(pipe, param_grid, scoring='f1', cv=sss)

In [65]:
grid = grid_search.fit(features_train,labels_train)

In [66]:
dump_classifier_and_data(grid_search.best_estimator_, my_dataset, features_list)
main()

Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('reducer', PCA(copy=True, iterated_power='auto', n_components=None, random_state=42,
  svd_solver='auto', tol=0.0, whiten=False)), ('selector', SelectKBest(k=2, score_func=<function f_classif at 0x1a17841ea0>)), ('classifier', GaussianNB(priors=None))])
	Accuracy: 0.85760	Precision: 0.44276	Recall: 0.26300	F1: 0.32999	F2: 0.28624
	Total predictions: 15000	True positives:  526	False positives:  662	False negatives: 1474	True negatives: 12338



>**Naive Bayes(Antes):** Accuracy: 0.84807	Precision: 0.38668	Recall: 0.23800	F1: 0.29465	F2: 0.25783

>**Naive Bayes(Depois):** Accuracy: 0.85760	Precision: 0.44276	Recall: 0.26300	F1: 0.32999	F2: 0.28624

### 5.3.2. Random Forest

In [67]:
pipe = Pipeline([
        ('scaler', preprocessing.StandardScaler()),
        ('reducer', PCA(random_state=42)),
        ('selector', SelectKBest()),
        ('classifier', RandomForestClassifier())
    ])

In [68]:
param_grid = { 
    'classifier__n_estimators': [10,20,40,80,160],
    'classifier__max_features': ['auto', 'sqrt', 'log2', None],
    'classifier__criterion': ['gini', 'entropy'],
    'selector__k': [2,4,6,8,10,12,14,16,18,20,'all']
}

In [69]:
sss = StratifiedShuffleSplit(n_splits=10, test_size=0.2, random_state=42)

In [70]:
grid_search = GridSearchCV(pipe, param_grid, scoring='f1', cv=sss)

In [71]:
grid = grid_search.fit(features_train,labels_train)

In [72]:
dump_classifier_and_data(grid_search.best_estimator_, my_dataset, features_list)
main()

Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('reducer', PCA(copy=True, iterated_power='auto', n_components=None, random_state=42,
  svd_solver='auto', tol=0.0, whiten=False)), ('selector', SelectKBest(k=2, score_func=<function f_classif at 0x1a17841ea0>)), ('classifi...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])
	Accuracy: 0.84080	Precision: 0.36621	Recall: 0.26550	F1: 0.30783	F2: 0.28095
	Total predictions: 15000	True positives:  531	False positives:  919	False negatives: 1469	True negatives: 12081



>**Random Forest(Antes):** Accuracy: 0.85913	Precision: 0.40989	Recall: 0.12850	F1: 0.19566	F2: 0.14895

>**Random Forest(Depois):** Accuracy: 0.84080	Precision: 0.36621	Recall: 0.26550	F1: 0.30783	F2: 0.28095

### 5.3.3. Decision Tree

In [73]:
pipe = Pipeline([
        ('scaler', preprocessing.StandardScaler()),
        ('reducer', PCA(random_state=42)),
        ('selector', SelectKBest()),
        ('classifier', tree.DecisionTreeClassifier())
    ])

In [74]:
param_grid = {
    'classifier__criterion': ['gini','entropy'],
    'classifier__splitter': ['best', 'random'],
    'classifier__min_samples_split': [2,4,8,16,32],
    'classifier__class_weight': ['balanced', None],
    'selector__k': [2,4,6,8,10,12,14,16,18,20,'all']
}

In [75]:
sss = StratifiedShuffleSplit(n_splits=10, test_size=0.2, random_state=42)

In [76]:
grid_search = GridSearchCV(pipe, param_grid, scoring='f1', cv=sss)

In [77]:
grid = grid_search.fit(features_train,labels_train)

In [78]:
dump_classifier_and_data(grid_search.best_estimator_, my_dataset, features_list)
main()

Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('reducer', PCA(copy=True, iterated_power='auto', n_components=None, random_state=42,
  svd_solver='auto', tol=0.0, whiten=False)), ('selector', SelectKBest(k=2, score_func=<function f_classif at 0x1a17841ea0>)), ('classifi...    min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='random'))])
	Accuracy: 0.81673	Precision: 0.30726	Recall: 0.29850	F1: 0.30282	F2: 0.30021
	Total predictions: 15000	True positives:  597	False positives: 1346	False negatives: 1403	True negatives: 11654



>**Decision Tree(Antes):** Accuracy: 0.79100	Precision: 0.23690	Recall: 0.25550	F1: 0.24585	F2: 0.25155

>**Decision Tree(Depois):** Accuracy: 0.81673	Precision: 0.30726	Recall: 0.29850	F1: 0.30282	F2: 0.30021

### 5.3.4. K-means

In [79]:
pipe = Pipeline([
        ('scaler', preprocessing.StandardScaler()),
        ('reducer', PCA(random_state=42)),
        ('selector', SelectKBest()),
        ('classifier', KMeans(n_clusters=2, random_state=0))
    ])

In [80]:
param_grid = {
    'classifier__algorithm': ['auto','full','elkan'],
    'classifier__random_state': [42],
    'classifier__precompute_distances': ['auto', True, False],
    'classifier__max_iter': [10,50,100,200,400,500],
    'classifier__n_clusters': [2],
    'selector__k': [2,4,6,8,10,12,14,16,18,20,'all']
}

In [81]:
sss = StratifiedShuffleSplit(n_splits=10, test_size=0.2, random_state=42)

In [82]:
grid_search = GridSearchCV(pipe, param_grid, scoring='f1',cv=sss)

In [83]:
grid = grid_search.fit(features_train,labels_train)

In [84]:
dump_classifier_and_data(grid_search.best_estimator_, my_dataset, features_list)
main()

Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('reducer', PCA(copy=True, iterated_power='auto', n_components=None, random_state=42,
  svd_solver='auto', tol=0.0, whiten=False)), ('selector', SelectKBest(k='all', score_func=<function f_classif at 0x1a17841ea0>)), ('classifier', KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=10,
    n_clusters=2, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=42, tol=0.0001, verbose=0))])
	Accuracy: 0.72767	Precision: 0.14064	Recall: 0.20400	F1: 0.16650	F2: 0.18714
	Total predictions: 15000	True positives:  408	False positives: 2493	False negatives: 1592	True negatives: 10507



>**K-means: Accuracy(Antes):** Accuracy: 0.85533	Precision: 0.31360	Recall: 0.07150	F1: 0.11645	F2: 0.08456

>**K-means: Accuracy(Depois):** Accuracy: 0.72767	Precision: 0.14064	Recall: 0.20400	F1: 0.16650	F2: 0.18714

### 5.3.5. Resultados Após Otimizações

>**Naive Bayes(Depois):** Accuracy: 0.85760 Precision: 0.44276 Recall: 0.26300 F1: 0.32999 F2: 0.28624

>**Random Forest(Depois):** Accuracy: 0.84080 Precision: 0.36621 Recall: 0.26550 F1: 0.30783 F2: 0.28095

>**Decision Tree(Depois):** Accuracy: 0.81673 Precision: 0.30726 Recall: 0.29850 F1: 0.30282 F2: 0.30021

>**K-means: Accuracy(Depois):** Accuracy: 0.72767	Precision: 0.14064	Recall: 0.20400	F1: 0.16650	F2: 0.18714

# 6. Conclusões

Neste caso de identificação de POI(Person of interest), ou seja, as pessoas que cometeram fraudes na emrpesa Eron, mas métricas mais significativas são a **precision** e a **recall**. 

O classificador que teve o melhor desempenho nos teste foi o **Decision Tree**, pois teve a métrica **F1** com maior valor. Essa métrica é a uma média harmonica entre precision e recall.

Os resultados do classificador são interpretados dessa forma:

- Recall ~30%: Quando uma pessoa que é POI é submetida ao classificador, 30% das vezes será classificado como POI. em outras palavras, 70% de Falsos Negativos (Erro tipo II).
- Precision ~30%: De todas as pessoas que são classificadas como POI, somente 30% são verdadeiros. Em outras palavras, temos 70% de Falsos positivos (Erro tipo I)